<a href="https://colab.research.google.com/github/Naeima/PoachNet/blob/main/Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install rdflib tensorflow scikit-learn pandas numpy

# Import necessary libraries
from rdflib import Graph
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input
from sklearn.metrics import mean_squared_error
import numpy as np
from google.colab import files



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 9.5 MB/s eta 0:00:00


In [9]:
import rdflib
from rdflib import URIRef, Literal, Graph, RDF, Namespace, plugin
from rdflib.plugins import sparql
from rdflib.plugins.sparql import prepareQuery
from rdflib.parser import Parser
import numpy as np
import networkx as nx
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from rdflib import Graph, URIRef, BNode, Literal
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

In [12]:
from google.colab import files
from rdflib import Graph
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Prompt the user to upload a file
uploaded = files.upload()

# Parse the uploaded RDF file
for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

    # Create an RDF graph
    g = Graph()

    # Attempt to parse the RDF file
    try:
        # Adjust the format as necessary (e.g., 'xml', 'turtle', 'ntriples', 'json-ld')
        g.parse(filename, format='turtle')
        print(f"Successfully parsed RDF file: {filename}")
    except Exception as e:
        print(f"Error parsing RDF file: {e}")

#  Define SPARQL query
query = prepareQuery("""
   SELECT * {
      ?s rdf:type <https://w3id.org/def/foo#gPSObservation>;
         <https://w3id.org/def/foo#localDate> ?LocalDate;
         <https://w3id.org/def/foo#localTime> ?localTime;
         <http://www.w3.org/2003/01/geo/wgs84_pos#latitude> ?lat;
         <http://www.w3.org/2003/01/geo/wgs84_pos#longitude> ?long .
   }
 """)


# Extract features and labels
features, labels = [], []
for row in g.query(query):
    try:
        long = float(row.long.toPython())
        lat = float(row.lat.toPython())
        local_date = str(row.LocalDate.toPython())
        local_time = str(row.localTime.toPython())

        # Convert date and time to numerical values (e.g., lengths)
        local_date_num = len(local_date)  # Example: Length of the date string
        local_time_num = len(local_time)  # Example: Length of the time string

        features.append([long, lat, local_date_num, local_time_num])
        labels.append([long, lat])
    except Exception as e:
        print(f"Skipping invalid row: {e}")

# Validate data before conversion
if not features or not labels:
    print("Error: No valid data extracted. Check your RDF query or data.")
    exit()

# Convert to NumPy arrays
try:
    features_np = np.array(features, dtype=np.float32)
    labels_np = np.array(labels, dtype=np.float32)
except ValueError as e:
    print(f"Error converting features or labels to NumPy arrays: {e}")
    exit()

# Split data
X_train_temp, X_test, y_train_temp, y_test = train_test_split(features_np, labels_np, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.25, random_state=42)

# Train model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,)),  # Features have 4 dimensions
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

history = model.fit(X_train, y_train, epochs=300, batch_size=32, validation_data=(X_val, y_val))

# Evaluate model
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {test_loss}, Test Mean Absolute Error: {test_mae}")

# Predict on test data
y_pred = model.predict(X_test)

# Calculate RMSE for longitude and latitude
rmse_long = np.sqrt(mean_squared_error(y_test[:, 0], y_pred[:, 0]))  # Longitude
rmse_lat = np.sqrt(mean_squared_error(y_test[:, 1], y_pred[:, 1]))  # Latitude

print(f"Test Longitude RMSE: {rmse_long}")
print(f"Test Latitude RMSE: {rmse_lat}")



Saving Seri(3).ttl to Seri(3) (6).ttl
User uploaded file "Seri(3) (6).ttl" with length 10482711 bytes
Successfully parsed RDF file: Seri(3) (6).ttl
Epoch 1/300
172/172 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 822.2344 - mean_absolute_error: 12.7259 - val_loss: 0.0053 - val_mean_absolute_error: 0.0548
Epoch 2/300
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0057 - mean_absolute_error: 0.0560 - val_loss: 0.0053 - val_mean_absolute_error: 0.0542
Epoch 3/300
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0061 - mean_absolute_error: 0.0568 - val_loss: 0.0069 - val_mean_absolute_error: 0.0575
Epoch 4/300
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0061 - mean_absolute_error: 0.0565 - val_loss: 0.0051 - val_mean_absolute_error: 0.0539
Epoch 5/300
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0065 - mean_absolute_error: 0.0577 - val_loss: 0.0056 - val_mean_absolute_error: 0.0547
Epoch 6/300
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0061 - mean_absolute_error: 0.0